In [76]:
%matplotlib inline
##from skimage import data, io, filters
import os
import numpy as np
from PIL import Image
import matplotlib
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd
import json
import pymongo as pm
import analysis_helpers as h
reload(h)

<module 'analysis_helpers' from 'analysis_helpers.pyc'>

#### setup and establish access to the database

In [97]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['3dObjects']
coll = db['shapenet_chairs_speaker_eval']

In [98]:
## get records
recs = coll.find({'$and': [{'iterationName':'pilot0'},{'timestamp':{'$gt':1507674252874}}]}).sort('gameID')
print '{} records in the database.'.format(recs.count())
print '{} unique workers participated.'.format(len(recs.distinct('wID')))

2832 records in the database.
46 unique workers participated.


#### loop through games and build up group data csv

In [99]:
# get list of games
recs = coll.find({'$and': [{'iterationName':'pilot0'},{'timestamp':{'$gt':1507674252874}}]}).sort('gameID')
games = recs.distinct('gameID')

In [ ]:
# extract relevant info from db
trialNum = []
condition = []
correct = []
family = []
gameID = []
response = []
rt = []
target = []
distractor1 = []
distractor2 = []

for i,this_game in enumerate(games):    
    print 'Analyzing {} of {} games ... '.format(i,len(games))
    game_recs = coll.find({'$and': [{'iterationName':'pilot0'},{'timestamp':{'$gt':1507674252874}},\
                                   {'gameID':this_game}]}).sort('timestamp')
    
    for trial in game_recs:
        trialNum.append(trial['trialNum'])
        condition.append(trial['condition'])
        correct.append(trial['correct'])
        family.append(trial['family'])
        gameID.append(trial['gameID'])
        response.append(trial['response'].split('_')[0])
        rt.append(trial['rt'])
        target.append([i.split('_')[0] for i in recs[0]['choices'] if i[-6:]=='target'][0])
        distractor1.append([i.split('_')[0] for i in recs[0]['choices'] if i[-11:]=='distractor1'][0])
        distractor2.append([i.split('_')[0] for i in recs[0]['choices'] if i[-11:]=='distractor2'][0])
            

Analyzing 0 of 54 games ... 
Analyzing 1 of 54 games ... 


In [ ]:
# make dataframe
X = pd.DataFrame([trialNum,condition,correct,family,gameID,response,rt,target,distractor1,distractor2])
X = X.transpose()
X.columns = ['trialNum','condition','correct','family','gameID','response','rt','target','distractor1','distractor2']
X = h.convert_numeric(X,'correct')
X = h.convert_numeric(X,'rt')

In [ ]:
# save out to csv
X.to_csv('./shapenet_chairs_speaker_eval_group_data.csv')

In [ ]:
# quick look at effect of condition (pragmatic vs. literal)
X.groupby('condition')['correct'].mean()

In [ ]:
# quick plot to look at difference in identification accuracy
sns.barplot(data=X,
            x='condition',
            y='correct')